In [ ]:
import sys
sys.path.extend([ '../', '../../'])
import torch
import dataloader as dl
from args import Args
from model import Het_ConEn, Het_NetEn, EdgePredictor, Edge_classify
from train2 import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.movielens_edge_pred()
torch.cuda.empty_cache()

In [ ]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('ml_data.pt')
print(data)

In [ ]:
print(data['u','walk','m'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['u_embed']['x'] = data['u_embed']['x'].to(device)
data['u_net_embed']['x'] = data['u_net_embed']['x'].to(device)
data['m_embed']['x'] = data['m_embed']['x'].to(device)
data['m_net_embed']['x'] = data['m_net_embed']['x'].to(device)

# Send all y tensors to the device
data['u', 'edge', 'm']['y'] = data['u', 'edge', 'm']['y'].to(device)
data['u', 'edge', 'm']['edge_index'] = data['u', 'edge', 'm']['edge_index'].to(device)

# Send all edge_index tensors to the device
data['u', 'walk', 'u']['edge_index'] = data['u', 'walk', 'u']['edge_index'].to(device)
data['u', 'walk', 'm']['edge_index'] = data['u', 'walk', 'm']['edge_index'].to(device)
data['m', 'walk', 'u']['edge_index'] = data['m', 'walk', 'u']['edge_index'].to(device)
data['m', 'walk', 'm']['edge_index'] = data['m', 'walk', 'm']['edge_index'].to(device)


edge_indices_list = [[ data['u', 'walk', 'u'].edge_index, data['u', 'walk', 'm'].edge_index],
                [ data['m', 'walk', 'u'].edge_index, data['m', 'walk', 'm'].edge_index]]
target_edge = data['u', 'edge', 'm']['edge_index'].transpose(0, 1)

In [ ]:
c_train_num = dl.train_num(data['u', 'edge', 'm']['y'], args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['u', 'edge', 'm']['y'], c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))

In [ ]:
encoder1 = Het_ConEn(args.embed_dim, args, args.dropout)
encoder2 = Het_NetEn(args.embed_dim, args.dropout)
classifier = Edge_classify(args.embed_dim, args.nclass, args.dropout)
decoder_uu = EdgePredictor(args.embed_dim)
decoder_um = EdgePredictor(args.embed_dim)
decoder_mu = EdgePredictor(args.embed_dim)
decoder_mm = EdgePredictor(args.embed_dim)
# encoder = torch.load('pretrained/encoder.pth')
# encoder2 = torch.load('pretrained/encoder2.pth')
# decoder_pa = torch.load('pretrained/decoder_pa.pth')
# decoder_pp = torch.load('pretrained/decoder_pp.pth')
# decoder_pv = torch.load('pretrained/decoder_pv.pth')
decoder_list1 = [decoder_uu, decoder_um]
decoder_list2 = [decoder_mu, decoder_mm]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list1: decoder.to(device)
for decoder in decoder_list2: decoder.to(device)

## Training Part

In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices_list, target_edge, encoder1, encoder2, classifier, decoder_list1, decoder_list2, train_idx, 
val_idx, test_idx, args, os_mode = 'edge_sm', train_mode = '')

# profiler.disable()

In [ ]:
# torch.save(decoder_list1[0], 'pretrained/decoder_uu.pth')
# torch.save(decoder_list1[1], 'pretrained/decoder_um.pth')
# torch.save(decoder_list2[0], 'pretrained/decoder_mu.pth')
# torch.save(decoder_list2[1], 'pretrained/decoder_mm.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')